In [19]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from PIL import Image

from datasets import imagenet
from nets import inception
from preprocessing import inception_preprocessing

from tensorflow.contrib import slim

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

image_size = inception.inception_resnet_v2.default_image_size
checkpoints_dir = '/home/src/goodsdl/train/23'



In [72]:
pre_graph = tf.Graph()
with pre_graph.as_default():
    image = tf.placeholder(dtype=tf.int8,shape=[None,None,3],name='input_image')
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.expand_dims(image, 0)
    image = tf.image.resize_bilinear(image, [image_size, image_size], align_corners=False)
    image = tf.squeeze(image, [0])
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0, name='output_image')

pre_sess = tf.Session(graph=pre_graph)

    
graph = tf.Graph()
with graph.as_default():    
    images = tf.placeholder(dtype=tf.float32,shape=[None,image_size,image_size,3],name='input_tensor')
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_resnet_v2_arg_scope()):
        logits, _ = inception.inception_resnet_v2(images, num_classes=156, is_training=False)
    probabilities = tf.nn.softmax(logits,name='detection_classes')
    
    init_fn = slim.assign_from_checkpoint_fn(
        tf.train.latest_checkpoint(checkpoints_dir),
        slim.get_model_variables('InceptionResnetV2'))
    
    sess = tf.Session()
    init_fn(sess)

INFO:tensorflow:Restoring parameters from /home/src/goodsdl/train/23/model.ckpt-21992


In [76]:
input_image_tensor = pre_graph.get_tensor_by_name('input_image:0')
output_image_tensor = pre_graph.get_tensor_by_name('output_image:0')
input_images_tensor = graph.get_tensor_by_name('input_tensor:0')
detection_classes = graph.get_tensor_by_name('detection_classes:0')
image_dir = '/home/src/goodsdl/train/23/step2/6925303721367'

filelist = [os.path.join(image_dir, image_name) for image_name in os.listdir(image_dir)]

images = [pre_sess.run(output_image_tensor,feed_dict={input_image_tensor:Image.open(fname)}) for fname in filelist]
# print(images[0].shape)

image_nps = np.array(images)
# print(image_nps.shape)

probabilities = sess.run(detection_classes,feed_dict={input_images_tensor:image_nps})
# print(probabilities)
for i in range(probabilities.shape[0]):
    sorted_inds = [j[0] for j in sorted(enumerate(-probabilities[i]), key=lambda x:x[1])]
    for k in range(5):
        index = sorted_inds[k]
        print('%d:Probability %0.2f%% => [%s]' % (i, probabilities[i][index] * 100, index))
    print('')

0:Probability 21.43% => [67]
0:Probability 10.66% => [82]
0:Probability 10.61% => [155]
0:Probability 8.76% => [103]
0:Probability 4.51% => [101]

1:Probability 20.78% => [155]
1:Probability 18.94% => [67]
1:Probability 8.48% => [95]
1:Probability 7.27% => [154]
1:Probability 4.87% => [20]

2:Probability 36.65% => [95]
2:Probability 18.61% => [155]
2:Probability 9.33% => [20]
2:Probability 5.63% => [101]
2:Probability 3.65% => [135]

3:Probability 22.29% => [155]
3:Probability 14.23% => [101]
3:Probability 7.11% => [82]
3:Probability 6.90% => [40]
3:Probability 5.09% => [20]

4:Probability 47.13% => [154]
4:Probability 10.92% => [107]
4:Probability 5.09% => [128]
4:Probability 3.84% => [112]
4:Probability 3.37% => [70]

5:Probability 31.35% => [154]
5:Probability 14.37% => [70]
5:Probability 11.78% => [128]
5:Probability 9.39% => [112]
5:Probability 5.95% => [85]

6:Probability 37.62% => [154]
6:Probability 10.48% => [128]
6:Probability 7.91% => [113]
6:Probability 6.49% => [107]
6:Pro